# ESCAPE Summer School 2021: Big data for big science #1

<img src="../pictures/spark_escape_logo.png" alt="alt text" width="1000"/>

## Before we start

- The material can be found in the GitHub repo of the school, folder `spark/`
- All notebooks and exercises can be run in a Docker container. See the `README.md`.
- This notebook is configured to run on Google Colab (see instructions inside notebooks).

If you are running this notebook on Google Colab, you need to install Spark before. As we are working in local mode (i.e. there is only one Java Virtual Machine which hosts both driver and executor threads) we can simply install it using `pip`:

In [ ]:
# Uncomment these lines if you are using Google Colab
!pip install pyspark==3.1.1

from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

## Context

The main goal of this lecture is to get familiar with Apache Spark, and in particular its Python API called PySpark in a scientific context. In this first notebook, we will introduce few Apache Spark functionalities of interest (and by no means complete!).

### Apache Spark

<img src="../pictures/spark_timeline.png" alt="alt text"/>

[Apache Spark](https://spark.apache.org/) is a cluster computing framework, that is a set of tools to perform computation on a network of many machines. Spark started in 2009 as a research project, and it had a huge success so far in the industry. It is based on the so-called **MapReduce** cluster computing paradigm, popularized by the Hadoop framework using implicit data parallelism and fault tolerance.

### When using Apache Spark?

<img src="../pictures/spark_logo.png" alt="alt text" width="200" align="right"/>

- When you have a LOT of data (static, or streaming)
- When you need to perform iterative analyses over a large dataset
- When you want to perform interactive data analysis over a large dataset

### Why using Apache Spark?

<img src="../pictures/spark_logo.png" alt="alt text" width="200" align="right"/>

- Unified analytics engine for large-scale data processing: all in one!
- Open source
- Large and active community of developers and users

### What are the main features?

<img src="../pictures/spark_logo.png" alt="alt text" width="200" align="right"/>

- Implicit data parallelism
- Easily scalable from a laptop to entire clusters
- Fault tolerance
- Functional programming model

<img src="../pictures/scala_logo.png" alt="alt text" width="200"/>

The core of Spark is written in Scala which is a general-purpose programming language that has been started in 2004 by Martin Odersky (EPFL). The language is inter-operable with Java and Java-like languages, and Scala executables run on the Java Virtual Machine (JVM). While Scala contains many ideas from functional programming, it is multi-paradigm, including functional programming, imperative programming, object-oriented programming and concurrent computing.

Spark provides many functionalities exposed through Scala/Python/Java/R API (Scala being the native one). As far as this lecture is concerned, we will use the Python API (called PySpark). But feel free to put your hands on Scala, it's worth it. For those interested, you can have a look at this [tutorial](https://gitlab.in2p3.fr/MaitresNageurs/QuatreNages/Scala) on Scala.

### Learning objectives for session 1

- Loading and distributing data with Spark SQL (Apache Spark Data Sources API)
- Exploring DataFrame & partitioning
- Manipulating Spark SQL built-in functions

## Distributed computing

<img src="../pictures/spark_cluster.png" alt="alt text"/>

### Spark cluster

Spark runs on clusters of machines, and typically in a cloud environment. In a simple deploy mode, a cluster is made of a driver node, connected via ssh to several worker nodes. Each worker is typically linked to a distributed file system (HDFS, Ceph, Amazon S3, ...). Two execution modes: _client_ or _cluster_ mode.

### Resource Scheduling

Resource scheduling and optimisation can be a difficult operation if done manually (the standalone mode of Spark). Alternatively, Spark can run on clusters managed by cluster managers such as YARN, Mesos, or even Kubernetes. See https://spark.apache.org/docs/latest/#launching-on-a-cluster.

### Running applications step-by-step

1. `Driver` contacts the `cluster manager` and requests for resources to launch the `Executors` inside the `Workers`.
2. The `cluster manager` launches the `Executors` on behalf of the `Driver`.
3. Once the `Executors` are launched, they establish a direct connection with the `Driver`.
4. The `Driver` determines the total number of `Tasks` by checking the `Lineage`.
5. The `Driver` creates the `Logical` and `Physical Plan`.
6. Once the `Physical Plan` is generated, Spark allocates the `Tasks` to the `Executors`.
7. `Task` runs on `Executor` and each `Task` upon completion returns the result to the `Driver`.
8. Finally, when all `Tasks` are completed, the program exits, and Spark releases all the resources from the `Cluster Manager`.

## Spark SQL and DataFrames

There are 4 main libraries in Apache Spark:
1. **SQL & DataFrames**
2. Streaming & Structured Streaming
3. MLlib (Machine Learning)
4. GraphX (graph)

For this lecture, we will focus on the SQL & DataFrames library. This library lets you query structured data inside Apache Spark programs, using either SQL or a familiar DataFrame API (à la Pandas). Note that the other libraries are also using the functionnalities of the SQL & DataFrames one (unified interface).

## Apache Spark Data Sources

### A tour of data formats

- Many data formats used in the context of Big Data: CSV (1978), XML (1996), JSON (2001), Thrift (2007), Protobuf (2008), Avro & SequenceFile (2009), Parquet (2013), ORC (2016), and the list goes on... 
- Some are _naively_ structured versus more complex and highly optimised for big data treatment (e.g. Parquet). 

Unfortunately those are often not the data formats typically chosen by the scientific community, e.g.: FITS (1981), HDF5 (1988) format, or ROOT (1995). 

### Connecting to Data Source

The data source API in Apache Spark belongs to the [Spark SQL module](https://spark.apache.org/sql/). Note that Spark Core has some simple built-in ways to read data from disk (binary or text), but Spark SQL is more complete and give you access to DataFrames directly. If you want to connect a specific data source with Apache Spark:

- [indirect] Access and distribute your files as binary streams (Spark does it natively), and decode the data on-the-fly within executors using third-party libraries --> _Easy to do, but poor performance._
- [native] Use a built-in or custom connector to access, distribute and decode the data natively --> _More challenging to write, but excellent performance._

### Connectors for scientific data formats?

By default, Spark cannot read natively most scientific formats like FITS, HDF5, or ROOT. There were several attempts though to build connectors: 

[1] Z. Zhang et al, Kira: Processing Astronomy Imagery Using Big Data Technology, DOI 10.1109/TBDATA.2016.2599926.    
[2] Peloton et al, FITS Data Source for Apache Spark, Computing and Software for Big Science (1804.07501). https://github.com/astrolabsoftware/spark-fits   
[3] Liu et al, H5spark: bridging the I/O gap between Spark and scientific data formats on HPC systems, Cray user group (2016). https://github.com/valiantljk/h5spark  
[4] Viktor Khristenko, & Jim Pivarski. (2017, October 20). diana-hep/spark-root: Release 0.1.14 (Version v0.1.14). Zenodo. http://doi.org/10.5281/zenodo.1034230

Most of scientific data formats, were not designed for serialisation (distribution of data over machines) originally and they often use compression to reduce the size on disk. Needless to say that default Spark cannot read those natively.

First attempts to connect those data formats (see e.g. [1] for FITS) with Spark were using the indirect method above. By reading files as binary streams, the indirect method has the advantage of having access to all FITS functionalities implemented in the underlying user library. This can be an advantage when working with the Python API for example which already contains many great scientific libraries. However this indirect method assumes each Spark mapper will receive and handle one entire file (since the filenames are parallelized and entire file data must be reconstructed from binary once the file has been opened by a Spark mapper). Therefore each single file must fit within the memory of a Spark mapper, hence the indirect method cannot distribute a dataset made of large FITS files (e.g. in [1] they have a 65 GB dataset made of 11,150 files). In addition by assuming each Spark mapper will receive and handle one entire file, the indirect method will have a poor load balancing if the dataset is made of files with not all the same size.

Fortunately Apache Spark low-level layers are sufficiently well written to allow extending the framework and write native connectors for any kind of data sources. Recently connectors for FITS, HDF5 and ROOT were made available [2, 3, 4] to the community. With such connectors, there is a guarantee of having a good load balancing regardless the structure of the dataset and the size of the input files is no more a problem (a 1 TB dataset made of thousand 1 GB files or one single 1 TB file will be viewed as almost the same by a native Spark connector). Note however that the Data Source API is in Java/Scala and if there is no library to play with your data source in those languages you must implement it (what has been done in [2]) or interface with another language.

Note that the low-level layers dealing with the data sources have been recently updated. Apache Spark 2.3 introduced the Data Source API version 2. While the version 1 is still available and usable for a long time, we expect that all Spark connectors will comply with this v2 in the future.

[1] Z. Zhang and K. Barbary and F. A. Nothaft and E. R. Sparks and O. Zahn and M. J. Franklin and D. A. Patterson and S. Perlmutter, Kira: Processing Astronomy Imagery Using Big Data Technology, DOI 10.1109/TBDATA.2016.2599926.  
[2] Peloton, Julien and Arnault, Christian and Plaszczynski, Stéphane, FITS Data Source for Apache Spark, Computing and Software for Big Science (1804.07501). https://github.com/astrolabsoftware/spark-fits   
[3] Liu, Jialin and Racah, Evan and Koziol, Quincey and Canon, Richard Shane, H5spark: bridging the I/O gap between Spark and scientific data formats on HPC systems, Cray user group (2016). https://github.com/valiantljk/h5spark  
[4] Viktor Khristenko, & Jim Pivarski. (2017, October 20). diana-hep/spark-root: Release 0.1.14 (Version v0.1.14). Zenodo. http://doi.org/10.5281/zenodo.1034230

## Spark hands-on: entry point

By default, this notebook comes with several instantiated objects to interact with the Spark cluster you just created by launching this notebook. The most important is the Spark Session:

In [ ]:
spark

Spark Session will allow you to read data, and manage the configuration of the cluster. Note you can also create it (or get the current one running) via

In [ ]:
from pyspark.sql import SparkSession

# Initialise our Spark session
spark = SparkSession.builder.getOrCreate()

### DataFrameReader

The interface to read data from disk is always the same for any kind of built-in and officially supported data format:

```python
df = spark.read\
    .format(format: str)\
    .option(key: str, value: Any)\
    # ...
    .option(key: str, value: Any)\
    .load(path: str)
```
 
Note that for most of the data sources, you can use wrappers such as:

```python
spark.read.csv(path, key1=value1, key2=value2, ...)
```

**Format**: The format can be "csv", "json", "parquet", etc. 

**Options**: The number of options depends on the underlying data source. Each has its own set of options. 
In most of the case, no options are needed, but you might want to explore the different possibilities at some point. Surprisingly it is not easy to find documentation and the best remains to read the source code documentation. In pyspark you can easily access it via the wrappers:

```python
# DataFrameReader object
df_reader = spark.read

# Doc on reading CSV
df_reader.csv?
# doc printed

# Doc on reading Parquet
df_reader.parquet?
# doc printed
```

**Path**: The way to specify path is threefold: either a single file (`path/to/folder/myfile.source`), or an entire folder (`path/to/folder`), or a glob pattern (`path/to/folder/*pattern*.source`). Note that you also need to specify the type of file system you are using. Example:

``` python
# Connect to hdfs
path = 'hdfs:///path/to/data'

# Connect to S3
path = 's3:///path/to/data'

# Connect to local file system
path = 'files:///path/to/data'
```

If nothing is specified (`'/path/to/data'`), it will adapt to your `--master` (e.g. if you launch spark in local mode, you will connect to the local file system by default).

## Loading and distributing data

You will find test data in the folder `data`.

### Some historical consideration

The main data object in Spark is the `DataFrame`. It was not always the case. Spark started with the concept of `RDD`. RDDs are distributed memory abstractions, fault-tolerant and immutable. They are designed for a general reuse (e.g. performing interactive data mining).

A `DataFrame` is a distributed collection of records, like a `RDD`, organized into named columns and including the benefits of Spark SQL’s execution engine. This is a new interface added in Spark version 1.6. Of course, the `RDD` interface is still accessible, and sometimes very useful.

### Using Data in Google colab

copy the content of https://github.com/gillardoLapp/school2021/tree/main/spark/data directory into your googledrive account under colabData directory

In [ ]:
# For google Colab

from google.colab import drive
drive.mount('/content/drive')

### Loading Data: simply structured data (text)

You can load CSV data into a DataFrame by simply using:

In [ ]:
# Load simple CSV file
df_csv_simple = spark.read.format("csv").load("/content/drive/MyDrive/colabData/clusters.csv")
df_csv_simple.printSchema()

Notice by default the CSV connector interprets all entries as String, and give dummy names to columns. 

**Exercise:** How would you infer the data type and use the first row as column names?

In [ ]:
# No schema information
df_csv_simple = spark.read.format("csv")\
    .load("/content/drive/MyDrive/colabData/clusters.csv")

# Add schema information
df_csv_with_schema = spark.read.format("csv")\
    .option('header', 'true')\
    .option('inferSchema', 'true')\
    .load("/content/drive/MyDrive/colabData/clusters.csv")

df_csv_with_schema.printSchema()

Let's have a look at the data

In [ ]:
df_csv_with_schema.show(10)

### Loading Data: complex structured data (Parquet)

More complex data format can infer automatically schema, and data types.
They are also optimised for fast data access and small memory consumption.

In [ ]:
# Note that the schema and the data types are directly inferred.
df_parquet = spark.read.format("parquet").load("/content/drive/MyDrive/colabData/clusters.parquet")

df_parquet.printSchema()

df_parquet.show(5)

### Loading Data: astronomy format (FITS)

To read FITS, you will need to specify a custom connector such as [spark-fits](https://github.com/astrolabsoftware/spark-fits) (this is done for you):

```bash
PYSPARK_DRIVER_PYTHON=jupyter-notebook pyspark --packages com.github.astrolabsoftware:spark-fits_2.12:0.9.0 ...
```

In [ ]:
# You need to specify the HDU
# Schema is automatically infered from the FITS header
df_fits = spark.read.format("fits").option("hdu", 1).load("/content/drive/MyDrive/colabData/clusters.fits")

## Data distribution

A DataFrame can be viewed as a distributed table. The table is chunked in `partitions` (i.e. one partition contains all columns, but only a subset of rows), and each Spark worker will process a subset of all partitions. When you load your data the first time, Spark will infer a default number of partitions based on several criteria, such as the type of file system (HDFS, S3, local, ...), or the initial partitioning of the dataset on disk (see next paragraph).

<img src="../pictures/spark_data_distribution.png" alt="alt text" width="1000"/>

**Exercise:** How are distributed the different DataFrames? In other words, how many partitions has each DataFrame?

In [ ]:
# write your answer here
print(df_csv_with_schema.rdd.getNumPartitions())

print(df_parquet.rdd.getNumPartitions())

print(df_fits.rdd.getNumPartitions())

Note how the partitioning varies as a function of the initial dataset.

## Partitioning

You might noticed Spark cut out the dataset into partitions, and for each partition Spark will run one task.
Following the principle that moving computation is usually cheaper than moving data, Spark reads file blocks in a performant way: instead of copying file blocks to a central compute node, which can be expensive, the driver sends the computation to worker nodes close to DataNodes where the data reside.
Normally, Spark tries to set the number of partitions automatically based on your distributed file system configuration. For example in HDFS, the size of data blocks is typically 128 MB (tunable), therefore the default number of Spark partitions when reading data will be the total number of 128 MB chunks for your dataset.

### How many partitions should I use?

There is no unique answer to that. You will often hear: `typically you want 2-4 partitions for each CPU in your cluster`, but that implies you can accomodate infinite number of CPUs at limited partition size. In practice it will mainly depend on: 
- the total volume of data you want to distribute, 
- the number of CPU you have access to and their RAM, 
- the kind of filesystem you are using,
- and the kind of task you want to perform.

&#9888; 

1. If you have too few partitions, you will not take benefit from all of the cores available in the cluster (time to solution can be longer, and you can run out of memory for intensive tasks). 
2. If you have too many partitions, there will be excessive overhead in managing many small tasks.

In between, you are generally good.

### How to repartition the data? (1/2)

You can repartition the dataset using:

```python
# numPartitions is arbitrary but
# this operation will add a shuffle step
df.repartition(numPartitions)

# Using either a number of partition or 
# column names to repartition by range
df.repartitionByRange(numPartitions, colnames)
```

### How to repartition the data? (2/2)

```python
# Repartition and sort
# using one or several columns
df.orderBy(colnames)

# numPartitions must be lower than the 
# current one, but no shuffle is performed
df.coalesce(numPartitions)
```

Frequent basic use-cases:
- The standard: You have a lot of data stored in large files and data entries need to be process independently from each other --> keep the default.
- The multi-files: When reading many small files (each being much smaller than the typical 128 MB data block size), you usually end up with way more partitions than if you were reading the same volume of data but with fewer files --> repartition your dataset with fewer partitions.
- The shuffle: If your tasks involve a lot of data movement and communication between machines (data shuffle) --> it is usually a good idea to keep the number of partitions not too high.
- The heavy filter: sometimes you filter out a lot of data based on some condition, and then you execute some action on the remaining subset. Because of the filering, you might end up with many empty partitions --> try to see if repartitioning with fewer partitions helps in processing the remaining faster.

**In practice you will end up experimenting a bit with the number of partitions... But always keep in mind the main reason to repartition is to minimize data movement inside the cluster.**

## Basic operations on DataFrames

Let's play a bit with our data

In [ ]:
# and let's load the Parquet dataset
df = spark.read.format("parquet").load("/content/drive/MyDrive/colabData/clusters.parquet")

### Select & filters

There are powerful methods to select subsets of columns or to filter rows based on values. Note that column selection and row filtering are transformations (in the sense of functional programming) - nothing really happens to the data until you trigger an action.

In [ ]:
# Filtering rows based on entry values
df.filter("x > 1")

# Same as before, but different syntax
df.filter(df["x"] > 1)

You can also filter on column names directly:

In [ ]:
# Filtering column based on their name
df_y_only = df.select('y')
df_x_and_y = df.select(['x', 'y'])

You can chain transformations:

In [ ]:
# You can chain transformations
df_x_cluster_one = df.filter('id == 1').select('x')

and finally trigger an action to start the computation:

In [ ]:
# count() is an "action"
row_with_x_more_than_one = df.filter("x > 1").count()

print("{} entries with x > 1".format(row_with_x_more_than_one))

### Mathematical functions

Many built-in functions are available in `pyspark.sql.functions`.

In [ ]:
import pyspark.sql.functions as F

df.select(
    [
        *df.columns,
        (F.sqrt(df['x']**2 + df['y']**2 + df['z']**2)).alias('radius')
    ]
).show(3)

In [ ]:
print(help(F))

### Statistics

You can easily access basics statistics of your DataFrame:

In [ ]:
# The describe method returns a DataFrame
df.describe().show()

### Aggregation

Apache Spark has built-in method to perform aggregation. Be careful though - this implies shuffle (i.e. communication between machines and data transfer), and can be a performance killer (naively, this scales quadratically in the number of partitions)!

<img src="../pictures/shuffle.png" alt="alt text" width="1000"/>

**Exercise:** group by `id`, and count the number of elements per `id`

In [ ]:
df.groupBy('id').count().show()

**Exercise:** Using `groupBy` and `agg`, compute the barycentre for each ID:

In [ ]:
df.groupBy('id').agg({'x': 'mean', 'y': 'mean', 'z': 'mean'}).show()

### Logical plan & physical plan

As quickly highlighted above, Spark commands are either transformations (filter, select, ...) or actions (show, take, ...). You can chain actions, and in the end you trigger the computation with an action. Before running any action, Spark will build a the graphs of the commands, called Direct Acyclic Graphs. One is the logical plan (what you wrote), the other is the physical plan (what will run). And... it will do some magic for you. 

**Exercise:** Look at the two commands and outputs. Do you notice the magic?

In [ ]:
df.filter('id >= 1').groupBy("id").count().explain(True)

In [ ]:
df.groupBy("id").count().filter('id >= 1').explain(True)

The DAG is a powerful tool to debug and optimize performance. Let's look at a common mistake: repartitioning

In [ ]:
# Let's fake a DataFrame with many partitions
# and let's cache the result on executors
df_tmp = df.repartition(32).cache()
df_tmp.count()

# Now, let's measure the time to repartition
# using `repartition` vs `coalesce`
%timeit df_tmp.repartition(2).count()
%timeit df_tmp.coalesce(2).count()

There are different by several factors!

In [ ]:
df.repartition(2).explain()

In [ ]:
df.coalesce(2).explain()

## Back to the driver

Once you have filtered, selected, and processed your data on the executors, you would like to return to the driver to finish your analysis (plotting, or perhaps just quick data mining). There are several ways of doing it:

In [ ]:
df_sub = df.filter('y < 0.5').filter('x > 1').select(['x', 'id'])

# Collect only 10 elements
some_rows = df_sub.take(10)

# Collect the data as a list of Rows
rows = df_sub.collect()

# Collect data as pandas DataFrame
pdf = df_sub.toPandas()

In [ ]:
pdf

WARNING: Use these methods only after reducing the data volume! These methods take the data left after processing on ALL the executors (it can be terabytes), and send it to the RAM of the driver. In case you do not know how much you've reduced, use `df_sub.count()` before.

## Acknowledgements

<img src="../pictures/logo-Escape_0.png" alt="alt text" width="400" align="right"/>

This event is organized in the framework and with the support of the European Science Cluster of Astronomy & Particle physics ESFRI research infrastructures (ESCAPE), funded by the European Union's Horizon 2020 - Grant N. 824064.